In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import microtool

%matplotlib widget

# Inversion recovery

## 1. Create a tissue model specifying a T1 and T2

In [ ]:
relaxation_model = microtool.tissue_model.RelaxationTissueModel(t1=900, t2=90)
print(relaxation_model)

## 2. Create an initial inversion-recovery acquisition scheme
Initial TR = 500 ms, initial TE = 10 ms, initial TI = {50, ..., 400} ms

In [ ]:
tr = np.array([500, 500, 500, 500, 500, 500, 500, 500])
te = np.array([10, 10, 10, 10, 20, 20, 20, 20])
ti = np.array([50, 100, 150, 200, 250, 300, 350, 400])

ir_scheme = microtool.acquisition_scheme.InversionRecoveryAcquisitionScheme(tr, te, ti)
print(ir_scheme)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(relaxation_model(ir_scheme), '.')
plt.xlabel('Measurement')
plt.ylabel('Signal attenuation');

## 3. Optimize the acquisition scheme

In [ ]:
noise_variance = 0.1
relaxation_model.optimize(ir_scheme, noise_variance,method=microtool.optimize.SOMA);

In [ ]:
print(ir_scheme)
plt.figure(figsize=(6, 4))
plt.plot(relaxation_model(ir_scheme), '.')
plt.xlabel('Measurement')
plt.ylabel('Signal attenuation');

# dmpyi diffusion model

In [ ]:
from dmipy.signal_models.cylinder_models import C1Stick
from dmipy.signal_models.gaussian_models import G1Ball
from dmipy.core.modeling_framework import MultiCompartmentModel

import microtool.dmipy

## 1. Create a 'stick' diffusion model

In [ ]:
dmipy_model = MultiCompartmentModel(models=[
    C1Stick(
        mu=[1, 1],  # Orientation in angles.
        lambda_par=0.001 * 1e-6  # Parallel diffusivity in m²/s.
    )
])

## 2. Wrap the dmipy model in a DmipyTissueModel

In [ ]:
diffusion_model = microtool.dmipy.DmipyTissueModel(dmipy_model)
diffusion_model

## 3. Create an initial diffusion acquisition scheme

In [ ]:
b_values = np.array([0, 1000, 2000, 3000])  # s/mm²
b_vectors = np.array([[0, 1, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
pulse_widths = np.full(b_values.shape, 10)  # ms
pulse_intervals = np.full(b_values.shape, 30)  # ms

diffusion_scheme = microtool.acquisition_scheme.DiffusionAcquisitionScheme(b_values, b_vectors, pulse_widths, pulse_intervals)
print(diffusion_scheme)

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(diffusion_model(diffusion_scheme), '.')
plt.xlabel('Measurement')
plt.ylabel('Signal attenuation');

## 4. Calculate the Cramer-Rao lower bound loss

In [ ]:
jacobian = diffusion_model.jacobian(diffusion_scheme)  # Jacobian of the signal with respect to the relevant tissue parameters.
scales = [p.scale for p in diffusion_model.values()]  # Tissue parameter scales.
include = [p.optimize for p in diffusion_model.values()]  # Include tissue parameter in optimization?
noise_variance = 0.1
microtool.optimize.crlb_loss(jacobian, scales, include, noise_variance)

## 5. Optimize the acquisition scheme

In [ ]:
diffusion_model.optimize(diffusion_scheme, noise_variance);

In [ ]:
print(diffusion_scheme)
plt.figure(figsize=(6, 4))
plt.plot(diffusion_model(diffusion_scheme), '.')
plt.xlabel('Measurement')
plt.ylabel('Signal attenuation');

## 6. Calculate the Cramer-Rao lower bound loss again
It should be lower after optimizing the acquisition.

In [ ]:
jacobian = diffusion_model.jacobian(diffusion_scheme)  # Jacobian of the signal with respect to the relevant tissue parameters.
scales = [p.scale for p in diffusion_model.values()]  # Tissue parameter scales.
include = [p.optimize for p in diffusion_model.values()]  # Include tissue parameter in optimization?
noise_variance = 0.1
microtool.optimize.crlb_loss(jacobian, scales, include, noise_variance)

# MISST diffusion model

## 1. Set the path to the MISST MATLAB package

In [ ]:
import microtool.misst
microtool.misst.set_misst_path(r'C:\development\MISST')

## 2. Create a 'Cylinder' diffusion model and wrap it in a MisstTissueModel

In [ ]:
misst_model = {
    'di': microtool.tissue_model.TissueParameter(value=2e-9, scale=1e-9),  # Intrinsic diffusivity in m²/s.
    'rad': microtool.tissue_model.TissueParameter(value=5.2e-6, scale=1e-6, optimize=False),  # Cylinder radius in m.
    'theta': microtool.tissue_model.TissueParameter(value=0.1, scale=1),  # Angle from z axis
    'phi': microtool.tissue_model.TissueParameter(value=0.2, scale=1),  # Azimuthal angle
}

In [ ]:
diffusion_model = microtool.misst.MisstTissueModel('Cylinder', misst_model)
print(diffusion_model)

## 3. Create an initial diffusion acquisition scheme

In [ ]:
b_values = np.array([0, 500, 1000, 1500, 2000, 2500, 3000])  # s/mm²
b_vectors = np.array([[0, 1, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1]])
pulse_widths = np.full(b_values.shape, 10)  # ms
pulse_intervals = np.full(b_values.shape, 30)  # ms

diffusion_scheme = microtool.acquisition_scheme.DiffusionAcquisitionScheme(b_values, b_vectors, pulse_widths, pulse_intervals)
print(diffusion_scheme)

## 4. Optimize the acquisition scheme

In [ ]:
noise_variance = 0.1
diffusion_model.optimize(diffusion_scheme, noise_variance);

In [ ]:
print(diffusion_scheme)
plt.figure(figsize=(6, 4))
plt.plot(diffusion_model(diffusion_scheme), '.')
plt.xlabel('Measurement')
plt.ylabel('Signal attenuation');